ST-GCN VIZULIZATION

In [ ]:

import os
import numpy as np
import torch
import torch.nn as nn
import json
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tabulate import tabulate
import joblib


BEST_MODEL_PATH = "/kaggle/working/best_stgcn_model_state_focused.pth"
BEST_SCALER_PATH = "/kaggle/working/best_stgcn_scaler_focused.joblib"
COMPONENTS_LIST = ["skating_skills", "transitions", "performance", "composition", "interpretation"]
OUTPUT_DIM = len(COMPONENTS_LIST)


SEQ_LEN = 1000
INCLUDE_VELOCITIES = True
INCLUDE_ACCELERATIONS = False
POSITION_ENCODING = "absolute"


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


NUM_JOINTS = 17  

adjacency_matrix = torch.zeros((NUM_JOINTS, NUM_JOINTS))

for i in range(NUM_JOINTS):
    adjacency_matrix[i, i] = 1

for i in range(NUM_JOINTS-1):
    adjacency_matrix[i, i+1] = 1
    adjacency_matrix[i+1, i] = 1


adjacency_matrix = adjacency_matrix.to(DEVICE)


class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, adjacency_matrix):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.adj = adjacency_matrix

        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / np.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def normalize_adjacency(self, adj):
        adj = adj + torch.eye(adj.size(0), device=adj.device)
        degree = torch.sum(adj, dim=1)
        degree_inv_sqrt = torch.pow(degree, -0.5)
        degree_inv_sqrt[torch.isinf(degree_inv_sqrt)] = 0.
        degree_matrix_inv_sqrt = torch.diag(degree_inv_sqrt)
        adj_normalized = torch.matmul(torch.matmul(degree_matrix_inv_sqrt, adj), degree_matrix_inv_sqrt)
        return adj_normalized

    def forward(self, input):
        support = torch.matmul(input.transpose(1, 2), self.weight).transpose(1, 2)
        output = torch.matmul(support, self.adj)
        if self.bias is not None:
            output = output + self.bias.view(1, -1, 1)
        return output

class STGCNBlock(nn.Module):
    def __init__(self, in_channels, spatial_out_channels, temporal_out_channels,
                 temporal_kernel_size, adjacency_matrix, use_residual=True, dropout=0.0):
        super(STGCNBlock, self).__init__()

        self.use_residual = use_residual
        self.spatial_out_channels = spatial_out_channels
        self.temporal_out_channels = temporal_out_channels

        normalized_adj = self.normalize_adjacency(adjacency_matrix)
        self.gcn = GraphConvolution(in_channels, spatial_out_channels, normalized_adj)

        padding = temporal_kernel_size // 2
        self.tcn = nn.Conv1d(spatial_out_channels, temporal_out_channels,
                             temporal_kernel_size, padding=padding)

        self.bn_spatial = nn.BatchNorm1d(spatial_out_channels)
        self.bn_temporal = nn.BatchNorm1d(temporal_out_channels)

        self.relu = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(dropout)

        if self.use_residual and in_channels != temporal_out_channels:
            self.residual_conv = nn.Conv2d(in_channels, temporal_out_channels, kernel_size=1)
        else:
            self.residual_conv = None

    def normalize_adjacency(self, adj):
        adj = adj + torch.eye(adj.size(0), device=adj.device)
        degree = torch.sum(adj, dim=1)
        degree_inv_sqrt = torch.pow(degree, -0.5)
        degree_inv_sqrt[torch.isinf(degree_inv_sqrt)] = 0.
        degree_matrix_inv_sqrt = torch.diag(degree_inv_sqrt)
        adj_normalized = torch.matmul(torch.matmul(degree_matrix_inv_sqrt, adj), degree_matrix_inv_sqrt)
        return adj_normalized

    def forward(self, x):
        residual = x

        N, C, V, T = x.size()
        x_gc = x.permute(0, 3, 1, 2).contiguous().view(N * T, C, V)
        x_gc = self.gcn(x_gc)
        x_gc = self.bn_spatial(x_gc)
        x_gc = self.relu(x_gc)
        x_gc = self.dropout(x_gc)

        x_tcn = x_gc.view(N, T, self.spatial_out_channels, V).permute(0, 2, 3, 1).contiguous()
        x_tcn = x_tcn.view(N * V, self.spatial_out_channels, T)

        x_tcn = self.tcn(x_tcn)
        x_tcn = self.bn_temporal(x_tcn)
        x_tcn = self.relu(x_tcn)
        x_tcn = self.dropout(x_tcn)

        x_out = x_tcn.view(N, V, self.temporal_out_channels, T).permute(0, 2, 1, 3).contiguous()

        if self.use_residual:
            if self.residual_conv is not None:
                residual = self.residual_conv(residual)
            x_out = x_out + residual

        return x_out

class STGCNRegressor(nn.Module):
    def __init__(self, input_dim, num_joints, seq_len, output_dim, adjacency_matrix, stgcn_params, fc_dropout):
        super().__init__()

        self.num_joints = num_joints
        self.seq_len = seq_len
        self.adj = adjacency_matrix

        initial_channels = input_dim // num_joints
        if input_dim % num_joints != 0:
            raise ValueError(f"Input dimension ({input_dim}) must be divisible by number of joints ({num_joints}) for STGCN.")

        
        num_blocks = stgcn_params.get('NUM_STGCN_BLOCKS', 2)
        spatial_channels_list = stgcn_params.get('STGCN_SPATIAL_CHANNELS', 64)
        temporal_channels_list = stgcn_params.get('STGCN_TEMPORAL_CHANNELS', 256)
        temporal_kernel_size = stgcn_params.get('STGCN_KERNEL_SIZE', 9)
        use_residual = stgcn_params.get('STGCN_USE_RESIDUAL', True)
        block_dropout = stgcn_params.get('STGCN_BLOCK_DROPOUT', 0.3)

        
        if not isinstance(spatial_channels_list, (list, tuple)):
            spatial_channels = [spatial_channels_list] * num_blocks
        else:
            spatial_channels = list(spatial_channels_list)
        
        if not isinstance(temporal_channels_list, (list, tuple)):
            temporal_channels = [temporal_channels_list] * num_blocks
        else:
            temporal_channels = list(temporal_channels_list)

     
        self.stgcn_blocks = nn.ModuleList()
        in_c = initial_channels

        for i in range(num_blocks):
            spatial_out_c = spatial_channels[i]
            temporal_out_c = temporal_channels[i]
            self.stgcn_blocks.append(
                STGCNBlock(in_c, spatial_out_c, temporal_out_c,
                           temporal_kernel_size, self.adj, use_residual, block_dropout)
            )
            in_c = temporal_out_c

     
        final_temporal_channels = temporal_channels[-1]
        self.pool = nn.AdaptiveAvgPool2d(1)
        pooled_fc_input_dim = final_temporal_channels

   
        self.fc = nn.Sequential(
            nn.Linear(pooled_fc_input_dim, pooled_fc_input_dim // 2),
            nn.ReLU(),
            nn.Dropout(fc_dropout),
            nn.Linear(pooled_fc_input_dim // 2, output_dim)
        )

    def forward(self, x):
        N, T, total_features_flat = x.size()
        total_features_per_joint = total_features_flat // self.num_joints
        if total_features_flat % self.num_joints != 0:
            raise ValueError(f"Input feature dimension {total_features_flat} not divisible by {self.num_joints} joints.")

        x = x.view(N, T, self.num_joints, total_features_per_joint)
        x = x.permute(0, 3, 2, 1).contiguous()

        for block in self.stgcn_blocks:
            x = block(x)

        pooled = self.pool(x)
        pooled = pooled.view(pooled.size(0), -1)

        return self.fc(pooled)

def load_and_process_skeleton(npz_file_path, seq_len=SEQ_LEN, include_velocities=INCLUDE_VELOCITIES, 
                             include_accelerations=INCLUDE_ACCELERATIONS, 
                             position_encoding=POSITION_ENCODING, scaler=None):
    """Load and process a single skeleton .npz file for inference"""
    try:
        data = np.load(npz_file_path)['reconstruction']  
    except Exception as e:
        print(f"Failed to load {npz_file_path}: {e}")
        return None
        
    T, J, C = data.shape
    print(f"Loaded skeleton with shape: {data.shape}")
    if T == 0 or C != 3:
        print(f"Invalid data shape: {data.shape}")
        return None
        
    data = data.astype(np.float32)  
    

    x_processed = data
    if position_encoding == 'relative_root':
        root_joint_index = 0
        root_coords = data[:, root_joint_index:root_joint_index+1, :]
        x_processed = data - root_coords
    

    x_flat_spatial = x_processed.reshape(T, J * C)
    

    feature_list = [x_flat_spatial]
    
    if include_velocities:
        if T > 1:
            velocities = np.diff(x_flat_spatial, axis=0, prepend=x_flat_spatial[0:1, :] * 0)
        else:
            velocities = np.zeros_like(x_flat_spatial, dtype=np.float32)
        feature_list.append(velocities)
    
    if include_accelerations:
        temp_velocities = np.diff(x_flat_spatial, axis=0, prepend=x_flat_spatial[0:1, :] * 0)
        if T > 2:
            accelerations = np.diff(temp_velocities, axis=0, prepend=temp_velocities[0:1, :] * 0)
        else:
            accelerations = np.zeros_like(x_flat_spatial, dtype=np.float32)
        feature_list.append(accelerations)
    

    x_combined_flat = np.concatenate(feature_list, axis=1)
    

    if scaler is not None and hasattr(scaler, 'mean_'):
        try:
            x_combined_flat = scaler.transform(x_combined_flat)
        except Exception as e:
            print(f"Error during scaling: {e}")
    

    if T < seq_len:
        pad_shape = (seq_len - T, x_combined_flat.shape[1])
        pad = np.zeros(pad_shape, dtype=np.float32)
        x_final = np.vstack([x_combined_flat, pad])
        print(f"Padded sequence from {T} to {seq_len} frames")
    else:
        x_final = x_combined_flat[:seq_len]
        print(f"Truncated sequence from {T} to {seq_len} frames")
    
    return x_final

def load_annotation(json_file_path):
    """Load component scores from a JSON annotation file"""
    try:
        with open(json_file_path, 'r') as f:
            annotation = json.load(f)
        
       
        components = {}
        for comp in COMPONENTS_LIST:
            if comp in annotation['program_component']:
                print()
                components[comp] = annotation['program_component'][comp]['score_of_pannel']
            else:
                print(f"Warning: Component {comp} not found in annotation")
                components[comp] = 0.0
                
        return components
    except Exception as e:
        print(f"Failed to load annotation from {json_file_path}: {e}")
        return {comp: 0.0 for comp in COMPONENTS_LIST}

def main():

    npz_file = "/kaggle/input/fine-fs-dataset/skeleton/skeleton/1.npz"  
    json_file = "/kaggle/input/fine-fs-dataset/annotation/annotation/1.json"  
    
    print(f"Loading model from {BEST_MODEL_PATH}")
    
    try:
 
        checkpoint = torch.load(BEST_MODEL_PATH, map_location=DEVICE)
        model_state = checkpoint.get('model_state_dict', checkpoint)  
        
        
        stgcn_params = {
            'NUM_STGCN_BLOCKS': 2,
            'STGCN_SPATIAL_CHANNELS': 64,
            'STGCN_TEMPORAL_CHANNELS': 256,
            'STGCN_KERNEL_SIZE': 9,
            'STGCN_USE_RESIDUAL': True,
            'STGCN_BLOCK_DROPOUT': 0.0 
        }
        fc_dropout = 0.0  
        
       
        features_per_joint = 3  # 
        if INCLUDE_VELOCITIES:
            features_per_joint += 3  
        if INCLUDE_ACCELERATIONS:
            features_per_joint += 3  
            
        input_dim = NUM_JOINTS * features_per_joint
        print(f"Input dimension: {input_dim} (features_per_joint={features_per_joint}, NUM_JOINTS={NUM_JOINTS})")
        
       
        scaler = None
        try:
            scaler = joblib.load(BEST_SCALER_PATH)
            print(f"Loaded scaler from {BEST_SCALER_PATH}")
        except Exception as e:
            print(f"Could not load scaler: {e}, proceeding without scaling")
        
        
        model = STGCNRegressor(
            input_dim=input_dim, 
            num_joints=NUM_JOINTS, 
            seq_len=SEQ_LEN, 
            output_dim=OUTPUT_DIM,
            adjacency_matrix=adjacency_matrix,
            stgcn_params=stgcn_params,
            fc_dropout=fc_dropout
        )
        
        
        model.load_state_dict(model_state)
        model.to(DEVICE)
        model.eval()
        
        print("Model loaded successfully")
        
        
        print(f"Processing skeleton file: {npz_file}")
        features = load_and_process_skeleton(
            npz_file,
            seq_len=SEQ_LEN,
            include_velocities=INCLUDE_VELOCITIES,
            include_accelerations=INCLUDE_ACCELERATIONS,
            position_encoding=POSITION_ENCODING,
            scaler=scaler
        )
        
        if features is None:
            print("Failed to process skeleton data")
            return
            
        
        print(f"Loading annotation file: {json_file}")
        true_components = load_annotation(json_file)
        
        
        features_tensor = torch.tensor(features).unsqueeze(0).float().to(DEVICE)  # Add batch dimension
        
        
        print("Running inference...")
        with torch.no_grad():
            predicted_scores = model(features_tensor)
            predicted_scores = predicted_scores.cpu().numpy()[0]  
        
        
        table_data = []
        for i, comp_name in enumerate(COMPONENTS_LIST):
            true_score = true_components.get(comp_name, "N/A")
            pred_score = predicted_scores[i]
            true_score = float(true_score)
            pred_score = float(pred_score)
            if isinstance(true_score, (int, float)) and isinstance(pred_score, (int, float)):
                diff = pred_score - true_score
                table_data.append([comp_name, f"{true_score:.2f}", f"{pred_score:.2f}", f"{diff:.2f}"])
            else:
                table_data.append([comp_name, true_score, f"{pred_score:.2f}", "N/A"])
        
        print("\nPrediction Results:")
        headers = ["Component", "True Score", "Predicted Score", "Difference"]
        print(tabulate(table_data, headers=headers, tablefmt="grid"))
        
        
        if all(comp in true_components for comp in COMPONENTS_LIST):
            true_array = np.array([true_components[comp] for comp in COMPONENTS_LIST])
            mse = np.mean((true_array - predicted_scores)**2)
            mae = np.mean(np.abs(true_array - predicted_scores))
            print(f"\nMean Squared Error: {mse:.4f}")
            print(f"Mean Absolute Error: {mae:.4f}")
            
            
            print("\nComparison with Best Model Test Set Metrics:")
            config_metrics = {
                "skating_skills": {"MSE": 79.590846, "MAE": 7.254685, "R2": -0.249512},
                "transitions": {"MSE": 78.176524, "MAE": 7.116168, "R2": -0.313296},
                "performance": {"MSE": 79.004186, "MAE": 7.189218, "R2": -0.305964},
                "composition": {"MSE": 85.275591, "MAE": 7.510861, "R2": -0.325712},
                "interpretation": {"MSE": 77.420197, "MAE": 7.202172, "R2": -0.311066}
            }
            
            comparison_table = []
            for comp in COMPONENTS_LIST:
                true_val = true_components.get(comp, 0)
                pred_val = predicted_scores[COMPONENTS_LIST.index(comp)]
                comp_mse = (true_val - pred_val)**2
                comp_mae = abs(true_val - pred_val)
                comparison_table.append([
                    comp, 
                    f"{comp_mse:.4f}", 
                    f"{config_metrics[comp]['MSE']:.4f}",
                    f"{comp_mae:.4f}", 
                    f"{config_metrics[comp]['MAE']:.4f}"
                ])
            
            comp_headers = ["Component", "Sample MSE", "Best Model MSE", "Sample MAE", "Best Model MAE"]
            print(tabulate(comparison_table, headers=comp_headers, tablefmt="grid"))
        
    except Exception as e:
        print(f"Error during inference: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()

Using device: cuda
Loading model from /kaggle/working/best_stgcn_model_state_focused.pth
Input dimension: 102 (features_per_joint=6, NUM_JOINTS=17)
Loaded scaler from /kaggle/working/best_stgcn_scaler_focused.joblib
Model loaded successfully
Processing skeleton file: /kaggle/input/fine-fs-dataset/skeleton/skeleton/1.npz
Loaded skeleton with shape: (4450, 17, 3)
Truncated sequence from 4450 to 1000 frames
Loading annotation file: /kaggle/input/fine-fs-dataset/annotation/annotation/1.json





Running inference...

Prediction Results:
+----------------+--------------+-------------------+--------------+
| Component      |   True Score |   Predicted Score |   Difference |
+================+==============+===================+==============+
| skating_skills |         6.46 |             -0.61 |        -7.07 |
+----------------+--------------+-------------------+--------------+
| transitions    |         6.54 |             -0.63 |        -7.17 |
+----------------+--------------+--------------

/tmp/ipykernel_31/3962972331.py:313: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



LSTM VIZULIZATION

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import json
import joblib
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate  


class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, lstm_dropout, fc_dropout, pooling_strategy='mean'):
        super().__init__()
        self.pooling_strategy = pooling_strategy
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, bidirectional=True,
                            dropout=lstm_dropout if num_layers > 1 else 0)

        fc_input_dim = hidden_dim * 2

        self.fc = nn.Sequential(
            nn.Linear(fc_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(fc_dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(fc_dropout),
            nn.Linear(hidden_dim // 2, output_dim)
        )

    def forward(self, x):
        outs, (hn, cn) = self.lstm(x)

        if self.pooling_strategy == 'mean':
            pooled = torch.mean(outs, dim=1)
        elif self.pooling_strategy == 'last':
            pooled = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim=1)
        else:
            pooled = torch.mean(outs, dim=1)

        return self.fc(pooled)


def inference_single(skeleton_npz_path, annotation_json_path, model_path, scaler_path, device='cuda'):
   
    
    if device == 'cuda' and not torch.cuda.is_available():
        print("CUDA not available. Using CPU instead.")
        device = 'cpu'
    
    device = torch.device(device)
    print(f"Using device: {device}")
    
    
    components_list = ['skating_skills', 'transitions', 'performance', 'composition', 'interpretation']
    
    
    try:
        scaler = joblib.load(scaler_path)
        print(f"Loaded scaler from {scaler_path}")
    except Exception as e:
        print(f"Error loading scaler: {e}")
        return None
    
    
    hidden_dim = 256  
    num_layers = 2
    lstm_dropout = 0.3
    fc_dropout = 0.4
    output_dim = len(components_list)
    pooling_strategy = 'last'
    
    
    try:
        skeleton_data = np.load(skeleton_npz_path)['reconstruction']
        print(f"Loaded skeleton data with shape: {skeleton_data.shape}")
    except Exception as e:
        print(f"Error loading skeleton data: {e}")
        return None
    
    
    try:
        with open(annotation_json_path, 'r') as f:
            annotation_data = json.load(f)
        
        
        actual_scores = {}
        for component in components_list:
            
            if component in annotation_data['program_component']:
                actual_scores[component] = annotation_data['program_component'][component]['score_of_pannel']
            else:
                print(f"Warning: Component '{component}' not found in annotation data")
                actual_scores[component] = float('nan')
        
        print(f"Loaded annotation data with scores: {actual_scores}")
    except Exception as e:
        print(f"Error loading annotation data: {e}")
        return None
    
    
    T, J, C = skeleton_data.shape
    
    
    x_coords = skeleton_data.reshape(T, J * C).astype(np.float32)
    
   
    if T > 1:
        velocities = np.diff(x_coords, axis=0)
        velocities = np.vstack([np.zeros((1, J * C), dtype=np.float32), velocities])
    else:
        velocities = np.zeros_like(x_coords, dtype=np.float32)
    
    x_combined = np.concatenate([x_coords, velocities], axis=1)
    
    
    if x_combined.shape[1] != scaler.mean_.shape[0]:
        print(f"Warning: Feature dimension mismatch. Data has {x_combined.shape[1]} features, scaler expects {scaler.mean_.shape[0]}.")
        return None
    
    x_scaled = scaler.transform(x_combined)
    
    
    seq_len = 1000  
    if T < seq_len:
        pad_shape = (seq_len - T, x_scaled.shape[1])
        pad = np.zeros(pad_shape, dtype=np.float32)
        x_final = np.vstack([x_scaled, pad])
    else:
        x_final = x_scaled[:seq_len]
    
    
    x_tensor = torch.from_numpy(x_final).unsqueeze(0).to(device)  
    
    
    try:
        input_dim = x_tensor.shape[2]  
        model = LSTMRegressor(input_dim, hidden_dim, num_layers, output_dim, 
                             lstm_dropout, fc_dropout, pooling_strategy).to(device)
        
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Loaded model from {model_path}")
        model.eval()
    except Exception as e:
        print(f"Error setting up model: {e}")
        return None
    
    
    with torch.no_grad():
        normalized_pred = model(x_tensor)
        normalized_pred = normalized_pred.cpu().numpy()[0]  
    
   
    target_means = np.array([5.0, 5.0, 5.0, 5.0, 5.0])  
    target_stds = np.array([1.0, 1.0, 1.0, 1.0, 1.0])   
    
    
    predictions = normalized_pred * target_stds + target_means
    
    
    results = {
        'Component': components_list,
        'Actual': [actual_scores.get(comp, float('nan')) for comp in components_list],
        'Predicted': predictions.tolist(),
        'Difference': [(actual_scores.get(comp, float('nan')) - pred) for comp, pred in zip(components_list, predictions)]
    }
    
    results_df = pd.DataFrame(results)
    
    return results_df


if __name__ == '__main__':
    # File paths
    skeleton_npz_path = '/kaggle/input/fine-fs-dataset/skeleton/skeleton/1.npz'  # Replace with actual file path
    annotation_json_path = '/kaggle/input/fine-fs-dataset/annotation/annotation/1.json'  # Replace with actual file path
    model_path = '/kaggle/input/fine-fs-dataset/best_lstm_model_state_focused.pth'
    scaler_path = '/kaggle/input/fine-fs-dataset/best_lstm_scaler_focused.joblib'
    
    
    if not os.path.exists(skeleton_npz_path):
        skeleton_npz_path = input("Enter path to skeleton NPZ file: ")
    
    if not os.path.exists(annotation_json_path):
        annotation_json_path = input("Enter path to annotation JSON file: ")
    
    
    results_df = inference_single(skeleton_npz_path, annotation_json_path, model_path, scaler_path)
    
    if results_df is not None:
        
        print("\n--- Inference Results ---")
        print(tabulate(results_df, headers='keys', tablefmt='fancy_grid', floatfmt=".2f"))
        
        
        mae = np.mean(np.abs(results_df['Difference'].values))
        rmse = np.sqrt(np.mean(np.square(results_df['Difference'].values)))
        
        print(f"\nOverall Mean Absolute Error: {mae:.4f}")
        print(f"Overall Root Mean Squared Error: {rmse:.4f}")

Using device: cuda
Loaded scaler from /kaggle/input/fine-fs-dataset/best_lstm_scaler_focused.joblib
Loaded skeleton data with shape: (4450, 17, 3)
Loaded annotation data with scores: {'skating_skills': 6.46, 'transitions': 6.54, 'performance': 6.64, 'composition': 6.64, 'interpretation': 6.75}
Loaded model from /kaggle/input/fine-fs-dataset/best_lstm_model_state_focused.pth

--- Inference Results ---
╒════╤════════════════╤══════════╤═════════════╤══════════════╕
│    │ Component      │   Actual │   Predicted │   Difference │
╞════╪════════════════╪══════════╪═════════════╪══════════════╡
│  0 │ skating_skills │     6.46 │        4.88 │         1.58 │
├────┼────────────────┼──────────┼─────────────┼──────────────┤
│  1 │ transitions    │     6.54 │        4.89 │         1.65 │
├────┼────────────────┼──────────┼─────────────┼──────────────┤
│  2 │ performance    │     6.64 │        4.83 │         1.81 │
├────┼────────────────┼──────────┼─────────────┼──────────────┤
│  3 │ composition  

/tmp/ipykernel_31/2294375543.py:144: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

